# SURFACE SM ANOMALIES FROM SMAP

https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP10KM_soil_moisture

## PART I: QUERY DATA FROM GEE SERVER

In [1]:
#installing dependencies
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib as mpl

In [2]:
#user select raw data destination folder
destination_folder = input('destination_folder: ')

destination_folder: D:\Users\aster\Desktop\dzud-types\hoof-dzud\SM\SM-repo


In [3]:
#plot wold map
Map = geemap.Map()

In [4]:
#upload shapefile of AOE
prefectures_shp = '../SM-repo/prefectures-v3.shp'
prefectures = geemap.shp_to_ee(prefectures_shp)
Map.addLayer(prefectures, {}, 'prefectures')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Input DATE, GEE Snippet and band. Can also modify data bounds and map colors. 

In [10]:
#user input start year 
#end year automatically set to most current data

start_date = str('2021-02-01')
end_date = str('2021-03-01')
    
dataset = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture')     .filterDate (start_date, end_date)     .map(lambda image: image.clip(prefectures)) 
soilMoisture = dataset.select('ssma')    
soilMoistureVis = {
    'min': -4,
    'max': 4,
    'palette': ['0300ff', '418504', 'efff07', 'efff07', 'ff0303']    }
Map.setCenter(106, 47, 4)
Map.addLayer(soilMoisture, soilMoistureVis, 'Soil Moisture')
Map

Map(bottom=1741.0, center=[47, 106], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

### Monthly Mean. Input desired variables.

In [11]:
monthly_ssm = soilMoisture.reduce(ee.Reducer.mean())
Map.addLayer(monthly_ssm, soilMoistureVis, 'monthly_ssma Mean')
Map

Map(bottom=1741.0, center=[47, 106], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [12]:
monthly_ssm

In [13]:
monthly_ssm.getInfo()

{'type': 'Image',
 'bands': [{'id': 'ssma_mean',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

### to TIF

##### Change file name:

In [14]:
feature = prefectures
roi = feature.geometry()
out_dir = os.path.join(os.path.expanduser('~'), destination_folder)
filename = os.path.join(out_dir, 'geemap_average_SMAP_SSMA_february2021.tif')
image = monthly_ssm
geemap.ee_export_image(image, filename=filename, scale=1000, region=roi)

Generating URL ...
Please wait ...
Data downloaded to D:\Users\aster\Desktop\dzud-types\hoof-dzud\SM\SM-repo\geemap_average_SMAP_SSMA_february2021.tif


### CONT. TO NOTEBOOK 2. TIF to CSV